### # Soft Prompt Tuning for ROS 2 Command Generation
Using Qwen2.5-Coder-1.5B-Instruct


In [ ]:
import torch
import torch.nn as nn
from transformers import AutoModelForCausalLM, AutoTokenizer


2. Configuration

In [ ]:
MODEL_NAME = "Qwen/Qwen2.5-Coder-1.5B-Instruct"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

N_PROMPT_TOKENS = 20
LR = 1e-4
EPOCHS = 150
MAX_NEW_TOKENS = 64

3. Training Data (Markdown + Code)

In [ ]:
train_data = [
    (
        "Move forward 2 meters",
        "ros2 topic pub /cmd_vel geometry_msgs/msg/Twist \"{linear: {x: 2.0}}\""
    ),
    (
        "Turn left 90 degrees",
        "ros2 service call /rotate_robot robot_msgs/srv/Rotate \"{angle: 1.57}\""
    ),
    (
        "Navigate to waypoint A",
        "ros2 action send_goal /navigate_to_pose nav2_msgs/action/NavigateToPose "
        "\"{pose: {header: {frame_id: 'map'}, pose: {position: {x: 5.0, y: 2.0}}}}\""
    )
]


4. Load Model & Tokenizer (Markdown + Code)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16 if DEVICE == "cuda" else torch.float32,
    device_map="auto"
)

# Freeze base model
for p in model.parameters():
    p.requires_grad = False

model.eval()

5. Soft Prompt Module (Markdown + Code)

In [ ]:
class SoftPrompt(nn.Module):
    def __init__(self, n_tokens, embedding_layer):
        super().__init__()

        init_prompt = embedding_layer.weight[:n_tokens].detach().clone()
        self.prompt_embeddings = nn.Parameter(init_prompt)

        print("Soft prompt shape:", self.prompt_embeddings.shape)

    def forward(self, batch_size):
        return self.prompt_embeddings.unsqueeze(0).expand(batch_size, -1, -1)


6. Initialize Soft Prompt (Markdown + Code)

In [ ]:
embedding_layer = model.get_input_embeddings()
print("Embedding table shape:", embedding_layer.weight.shape)

soft_prompt = SoftPrompt(
    N_PROMPT_TOKENS,
    embedding_layer
).to(embedding_layer.weight.device)


7. Loss Function (Clean Version) (Markdown + Code)

In [ ]:
def compute_loss(input_text, target_text):
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(model.device)
    target_ids = tokenizer(target_text, return_tensors="pt").input_ids.to(model.device)
    batch_size = input_ids.size(0)

    # Token embeddings
    full_ids = torch.cat([input_ids, target_ids], dim=1)
    token_embeds = model.get_input_embeddings()(full_ids)

    # Soft prompt embeddings
    prompt_embeds = soft_prompt(batch_size)

    # Final embeddings
    full_embeds = torch.cat([prompt_embeds, token_embeds], dim=1)

    # Attention mask
    attention_mask = torch.ones(
        full_embeds.size()[:-1],
        device=model.device,
        dtype=torch.long
    )

    # Labels (ignore prompt + input)
    labels = torch.cat([
        torch.full(
            (batch_size, N_PROMPT_TOKENS + input_ids.size(1)),
            -100,
            device=model.device
        ),
        target_ids
    ], dim=1)

    outputs = model(
        inputs_embeds=full_embeds,
        attention_mask=attention_mask,
        labels=labels
    )
    return outputs.loss


8. Training Loop (Markdown + Code)

In [ ]:
optimizer = torch.optim.AdamW(soft_prompt.parameters(), lr=LR)

print("=" * 40)
print("🚀 Starting Prompt Tuning")
print("=" * 40)

for epoch in range(EPOCHS):
    total_loss = 0.0

    for inp, out in train_data:
        loss = compute_loss(inp, out)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(soft_prompt.parameters(), 1.0)
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1:03d} | Loss: {total_loss:.4f}")

print("✨ Training Complete!")


9. Save Soft Prompt (Markdown + Code)

In [ ]:
torch.save(soft_prompt.state_dict(), "soft_prompt_ros2.pt")
print("✅ Saved soft_prompt_ros2.pt")


10. Inference Function (Markdown + Code)

In [ ]:
def infer_ros2_command(human_input):
    input_ids = tokenizer(human_input, return_tensors="pt").input_ids.to(model.device)
    input_embeds = model.get_input_embeddings()(input_ids)

    prompt_embeds = soft_prompt(1)
    full_embeds = torch.cat([prompt_embeds, input_embeds], dim=1)

    with torch.no_grad():
        output_ids = model.generate(
            inputs_embeds=full_embeds,
            max_new_tokens=MAX_NEW_TOKENS,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )

    return tokenizer.decode(output_ids[0], skip_special_tokens=True)


11. Test the Model (Markdown + Code)

In [ ]:
tests = [
    "Move forward 9 meters",
    "Turn left 90 degrees",
    "Navigate to waypoint A"
]

for t in tests:
    print("Input :", t)
    print("Output:", infer_ros2_command(t))
    print("-" * 80)
